In [1]:
import pandas as pd
games=pd.read_csv("games.csv")
import pandas as pd

# Read the CSV file into a DataFrame
games = pd.read_csv("games.csv")

In [2]:
import pandas as pd

# Assuming you have a DataFrame called 'games' with columns 'Developers' and 'Publishers'

# Create a new column that indicates whether the Publisher is different from the Developer
games['Publisher_Differs'] = games['Publishers'] != games['Developers']

# Count the number of times Publisher is different from Developer
count_different = games['Publisher_Differs'].sum()

# Display the result
print("Count of times Publisher is different from Developer:", count_different)



Count of times Publisher is different from Developer: 24341


In [3]:
#!pip install annoy

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Assuming you have a DataFrame called 'games' with columns 'Name', 'Publishers', and 'Genres'

# Preprocess the data
games['Publishers'] = games['Publishers'].fillna('')
games['Genres'] = games['Genres'].fillna('')
games['Features'] = games['Publishers'] + ' ' + games['Genres']

# Create a TF-IDF vectorizer to convert text data into numerical features for both genres and combined features
tfidf_vectorizer_combined = TfidfVectorizer()
tfidf_matrix_combined = tfidf_vectorizer_combined.fit_transform(games['Features'])

# Create a TF-IDF vectorizer for genres only
tfidf_vectorizer_genres = TfidfVectorizer()
tfidf_matrix_genres = tfidf_vectorizer_genres.fit_transform(games['Genres'])

# Build a nearest neighbors model for both combined features and genres
knn_combined = NearestNeighbors(n_neighbors=6, metric='cosine')
knn_genres = NearestNeighbors(n_neighbors=6, metric='cosine')

knn_combined.fit(tfidf_matrix_combined)
knn_genres.fit(tfidf_matrix_genres)

# Function to recommend similar games
def recommend_similar_games(game_name):
    # Find the index of the given game in the DataFrame
    game_index = games[games['Name'] == game_name].index[0]
    
    # Use the nearest neighbors model to find the most similar games for both combined and genres models
    distances_combined, indices_combined = knn_combined.kneighbors(tfidf_matrix_combined[game_index])
    distances_genres, indices_genres = knn_genres.kneighbors(tfidf_matrix_genres[game_index])
    
    # Exclude the input game itself from recommendations for both models
    similar_game_indices_combined = indices_combined[0][1:]
    similar_game_indices_genres = indices_genres[0][1:]
    
    # Get the 5 most similar games for both models
    similar_games_combined = [games.loc[i]['Name'] for i in similar_game_indices_combined]
    similar_games_genres = [games.loc[i]['Name'] for i in similar_game_indices_genres]
    
    # Check if the priority model (combined) has enough recommendations
    if len(similar_games_combined) >= 5:
        return similar_games_combined[:5]
    else:
        # Combine recommendations from the priority model and genres model
        combined_recommendations = similar_games_combined + similar_games_genres
        
        # Remove duplicates and return up to 5 recommendations
        unique_combined_recommendations = list(set(combined_recommendations))
        return unique_combined_recommendations[:5]

# Example usage
input_game_name = "Galactic Bowling"
recommended_games = recommend_similar_games(input_game_name)

print(f"For the game '{input_game_name}', 5 similar recommendations are:")
for i, game in enumerate(recommended_games):
    print(f"{i+1}. {game}")


For the game 'Galactic Bowling', 5 similar recommendations are:
1. Football Club Simulator - FCS #21
2. Imperivm RTC - HD Edition Great Battles of Rome
3. Arena
4. Time Keeper
5. Dragon Lords: 3D Strategy


In [ ]:
games

In [ ]:
games

In [ ]:
editgames=pd.read_csv("editGame.csv")
editgames